# SRP - Single Responsibility Principle

**Klasa powinna mieć tylko jeden powód do zmiany.**

Innymi słowy: **Jedna klasa = jedna odpowiedzialność.**

## Złe rozwiązanie - łamie SRP

In [1]:
class Journal:
    def __init__(self):
        self.entries = []
        self.count = 0
    
    # Odpowiedzialność 1: Zarządzanie wpisami - ok
    def add_entry(self, text):
        self.count += 1
        self.entries.append(f"{self.count}: {text}")
    
    def remove_entry(self, pos):
        del self.entries[pos]
    
    def __str__(self):
        return "\n".join(self.entries)
    
    # Odpowiedzialność 2: Zapis do pliku - źle
    def save(self, filename):
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(str(self))
    
    # Odpowiedzialność 3: Ładowanie z pliku - źle
    def load(self, filename, encoding='utf-8'):
        with open(filename, 'r') as f:
            # ...
            pass
    
    # Odpowiedzialność 4: Ładowanie z sieci - źle
    def load_from_web(self, uri):
        # ...
        pass


In [2]:
# client code
j = Journal()
j.add_entry("Dziś było słonecznie")
j.add_entry("Zjadłem pizzę")
print(j)

j.save("journal.txt")
print("\nZapisano do pliku")

1: Dziś było słonecznie
2: Zjadłem pizzę

Zapisano do pliku


### Problem:

`Journal` ma **4 odpowiedzialności:**
1. Zarządzanie wpisami
2. Zapis do pliku
3. Ładowanie z pliku
4. Ładowanie z sieci

**Co jeśli:**
- Zmienimy format zapisu (JSON zamiast TXT)? → Edycja `Journal`
- Dodamy inne klasy (`Book`, `Article`)? → Duplikacja logiki zapisu
- Chcemy sprawdzić prawa zapisu? → Edycja WSZYSTKICH klas

**Anti-pattern:** God Object (klasa robi wszystko)

## Dobre rozwiązanie - zgodne z SRP

Wydziel odpowiedzialność za trwałość do osobnej klasy.

In [3]:
# Klasa 1: Tylko zarządzanie wpisami
class Journal:
    def __init__(self):
        self.entries = []
        self.count = 0
    
    def add_entry(self, text):
        self.count += 1
        self.entries.append(f"{self.count}: {text}")
    
    def remove_entry(self, pos):
        del self.entries[pos]
    
    def __str__(self):
        return "\n".join(self.entries)


In [4]:
# Klasa 2: Tylko trwałość (zapis/ładowanie)
class PersistenceManager:
    @staticmethod
    def save_to_file(obj, filename):
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(str(obj))
    
    @staticmethod
    def load_from_file(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()


In [5]:
# client code
j = Journal()
j.add_entry("Dziś było słonecznie")
j.add_entry("Zjadłem pizzę")
print("Dziennik:")
print(j)

# Zapis - oddzielna odpowiedzialność
PersistenceManager.save_to_file(j, "journal2.txt")
print("\nZapisano do pliku")

# Odczyt
content = PersistenceManager.load_from_file("journal2.txt")
print("\nZawartość pliku:")
print(content)

Dziennik:
1: Dziś było słonecznie
2: Zjadłem pizzę

Zapisano do pliku

Zawartość pliku:
1: Dziś było słonecznie
2: Zjadłem pizzę


## Porównanie

| Aspekt | Złe (God Object) | Dobre (SRP) |
|--------|---------------------|----------------|
| Odpowiedzialności | 4 w jednej klasie | 1 w każdej klasie |
| Zmiana formatu zapisu | Edytuj `Journal` | Edytuj `PersistenceManager` |
| Nowa klasa (`Book`) | Duplikuj logikę zapisu | Użyj `PersistenceManager` |
| Testowanie | Trudne (wszystko razem) | Łatwe (osobno) |
| Powodów do zmiany | 4 | 1 |

## Esencja SRP

### Pytanie:
Ile powodów do zmiany ma ta klasa?

### Złe (łamie SRP):
```python
class Journal:
    def add_entry(...):     # Powód 1: zmiana logiki wpisów
    def save(...):          # Powód 2: zmiana formatu zapisu
    def load(...):          # Powód 2: zmiana formatu zapisu
    def load_from_web(...): # Powód 3: zmiana API sieci
```
**3 powody** = 3 odpowiedzialności

### Dobre (zgodne z SRP):
```python
class Journal:
    def add_entry(...):  # Powód 1: zmiana logiki wpisów

class PersistenceManager:
    def save(...):       # Powód 2: zmiana formatu zapisu
```
**1 powód** w każdej klasie

## Jak rozpoznać łamanie SRP?

### Sygnały ostrzegawcze:
1. Klasa robi "X **i** Y" (nie "X")
2. Metody operują na różnych zbiorach danych
3. Zmiana w jednym obszarze wymaga edycji klasy
4. Trudno nazwać klasę jednym rzeczownikiem

### Przykłady:
```python
# Złe nazwy (wiele odpowiedzialności)
class UserManagerAndEmailSender:  # robi 2 rzeczy
class DataProcessorAndLogger:     # robi 2 rzeczy

# Dobre nazwy (jedna odpowiedzialność)
class UserManager:     # zarządza użytkownikami
class EmailSender:     # wysyła email
class DataProcessor:   # przetwarza dane
class Logger:          # loguje
```

## Podsumowanie

### SRP w jednym zdaniu:
**Jedna klasa = jedna odpowiedzialność = jeden powód do zmiany.**

### Jak stosować SRP?
1. Dla każdej klasy zapytaj: "Co robi?"
2. Jeśli odpowiedź zawiera "i" → rozdziel na dwie klasy
3. Każda zmiana powinna dotyczyć tylko jednej klasy

### Korzyści:
- Łatwiejsze testowanie (mniejsze klasy)
- Mniej konfliktów w zespole (różne osoby = różne klasy)
- Łatwiejsza rozbudowa (dodaj klasę, nie edytuj istniejącej)
- Wyższa spójność kodu

### Anti-pattern:
**God Object** - klasa robi wszystko, ma dziesiątki metod i setek linii.